In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.

# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# raries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models


# This code was tested with TensorFlow v1.8
print("You have TensorFlow version", tf.__version__)

In [ ]:
import pandas as pd
bbc_text = pd.read_csv("../input/bbc-fulltext-and-category/bbc-text.csv")

In [ ]:
bbc_text.head()
data = bbc_text

In [ ]:
data.head()

In [ ]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

df = data
df = df[pd.notnull(df['category'])]
print(df.head(10))
print(df['text'].apply(lambda x: len(x.split(' '))).sum())

In [ ]:
data['category'].value_counts()


In [ ]:
my_tags = [cat for cat in data['category'].unique()]
plt.figure(figsize=(10,4))
df.category.value_counts().plot(kind='bar')

In [ ]:
def print_plot(index):
    example = df[df.index == index][['text', 'category']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])
print_plot(10)

In [ ]:
print_plot(30)

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
#     text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['text'] = df['text'].apply(clean_text)
print_plot(10)

In [ ]:
df['text'].apply(lambda x: len(x.split(' '))).sum()

In [ ]:
X = df.text
y = df.category
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:

from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

%time

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

%time

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

# **Load word embedding pretrained on word Google news corpus**

In [ ]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"


In [ ]:
from gensim.models import Word2Vec
wv = gensim.models.KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
wv.init_sims(replace=True)

In [ ]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.vectors_norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [ ]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    
train, test = train_test_split(df, test_size=0.3, random_state = 42)

test_tokenized = test.apply(lambda r: w2v_tokenize_text(r['text']), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r['text']), axis=1).values

X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(X_train_word_average, train['category'])
y_pred = logreg.predict(X_test_word_average)
print('accuracy %s' % accuracy_score(y_pred, test.category))
print(classification_report(test.category, y_pred,target_names=my_tags))

In [ ]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

train_size = int(len(df) * .7)
train_posts = df['text'][:train_size]
train_tags = df['category'][:train_size]

test_posts = df['text'][train_size:]
test_tags = df['category'][train_size:]

max_words = 5000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 10

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.2)

In [ ]:
data_list = data['text']

In [ ]:
import nltk 
import string 
import re 

def text_lowercase(text): 
    return text.lower()

# Remove numbers 
def remove_numbers(text): 
    result = re.sub(r'\d+', '', text) 
    return result 

# remove punctuation 
def remove_punctuation(text): 
    translator = str.maketrans('', '', string.punctuation) 
    return text.translate(translator) 

# remove whitespace from text 
def remove_whitespace(text): 
    return  " ".join(text.split()) 

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
# remove stopwords function 
def remove_stopwords(text): 
    stop_words = set(stopwords.words("english")) 
    word_tokens = word_tokenize(text) 
    filtered_text = [word for word in word_tokens if word not in stop_words] 
    return filtered_text 
  
from nltk.stem.porter import PorterStemmer 
from nltk.tokenize import word_tokenize 
stemmer = PorterStemmer() 
  
# stem words in the list of tokenised words 
def stem_words(text): 
    word_tokens = word_tokenize(text) 
    stems = [stemmer.stem(word) for word in word_tokens] 
    return stems 



from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
lemmatizer = WordNetLemmatizer() 
# lemmatize string 
def lemmatize_word(text): 
    word_tokens = word_tokenize(text) 
    # provide context i.e. part-of-speech 
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens] 
    return lemmas 


def preprocessing(text):
    result = text_lowercase(text)
#     result = remove_numbers(result)
#     result = remove_punctuation(result)
#     result = remove_whitespace(result)
    result = remove_stopwords(result)
    
#     result = stem_words(result)
#     result = lemmatize_word(result)
    
    
    return result


In [ ]:
#using data_list we can preprocess the sentences here
import time
sentences = [sentence for sentence in data_list]
print(len(sentences))
start_time  = time.time()
preprocessed_sentences = [preprocessing(sentence) for sentence in sentences]
end_time = time.time()
print("total time taken in preprocessing {}".format(end_time-start_time))

In [ ]:
print(len(preprocessed_sentences))
print(len(sentences[4]))
print(len(preprocessed_sentences[4]))
print(sentences[4])
print(preprocessed_sentences[4])

In [ ]:
# dictionary for words according to frequency
from collections import defaultdict
word_freq = defaultdict(int)

for sent in preprocessed_sentences:
    for i in sent:
        word_freq[i] += 1
print(len(word_freq))

word_freq['subject']
most_freq_words = sorted(word_freq, key=word_freq.get, reverse=True)[:2000]

In [ ]:
vocabulary = [(key,value) for key,value in word_freq.items() if(value>=5)]

In [ ]:

print(len(vocabulary))
vocabulary[10]

def Convert(tup, di): 
    for a, b in tup: 
        di.setdefault(a, []).append(b) 
    return di 

vocabulary_dict = {}
vocabulary_dict = Convert(vocabulary,vocabulary_dict)
print(len(vocabulary_dict))

In [ ]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"


In [ ]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
x = word2vec.word_vec("test")

In [ ]:
import multiprocessing

from gensim.models import Word2Vec

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

cores = multiprocessing.cpu_count()

print(cores)

In [ ]:
from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec


class EpochSaver(CallbackAny2Vec):
#      '''Callback to save model after each epoch.'''

    def __init__(self, path_prefix):
        self.path_prefix = path_prefix
        self.epoch = 0

    def on_epoch_end(self, model):
        output_path = get_tmpfile('{}_epoch{}.model'.format(self.path_prefix, self.epoch))
        model.save(output_path)
        self.epoch += 1

In [ ]:
class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
   
    def on_epoch_begin(self, model):
         print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

epoch_logger = EpochLogger()
# w2v_model = Word2Vec(common_texts, iter=5, size=10, min_count=0, seed=42, callbacks=[epoch_logger])

In [ ]:
from gensim.models import Word2Vec
import time
w2v_model = Word2Vec(min_count=5,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=4)

start = time.time()
w2v_model.build_vocab(preprocessed_sentences, progress_per=10000/2)
total_examples = w2v_model.corpus_count
print(total_examples)

w2v_model.build_vocab([list(word2vec.vocab.keys())], update=True)
w2v_model.intersect_word2vec_format(EMBEDDING_FILE, binary =True, encoding='utf8')
w2v_model.train(preprocessed_sentences,total_examples=total_examples,epochs=30,callbacks=[epoch_logger])

print("total time taken {}".format(time.time()-start))


w2v_model.save('fine_tuned_30epochs_bbc_model')


In [ ]:
import os
os.getcwd()

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
w2v_model = Word2Vec.load("fine_tuned_30epochs_bbc_model")
vector  = w2v_model.wv['news']
print(vector.shape)
w2v_model = w2v_model.wv

In [ ]:
# vector

In [ ]:
import time
start_time  = time.time()
preprocessed_sentences = [preprocessing(sentence) for sentence in sentences]
end_time = time.time()
print("total time taken in preprocessing {}".format(end_time-start_time))

In [ ]:
start = time.time()
data_matrix = []
for i in range(len(preprocessed_sentences)):
    word_dict ={}
    for index,j in enumerate(preprocessed_sentences[i]):
            if(j not in word_dict):
                word_dict[j]=1
            else:
                word_dict[j]+=1
    arr = []
    for word in w2v_model.vocab:
        if(word in word_dict):
            arr.append(word_dict[word])
        else:
            arr.append(0)
        
    
    data_matrix.append(arr)
end = time.time()
print('total time taken in building matrix {}'.format(end-start))


In [ ]:
data_matrix = np.array(data_matrix)
print(data_matrix.shape)
print(data_matrix.dtype)
data_matrix = data_matrix.astype('int32')
np.save('data_matrix',data_matrix)

In [ ]:
data_matrix = np.load('data_matrix.npy')

In [ ]:
len(most_freq_words)
most_freq_words[101]

In [ ]:
H_matrix = []
start = time.time()
for i in w2v_model.vocab:
    arr = []
    for word in most_freq_words:
        similar = w2v_model.similarity(word, i)
        if(similar>0):
            arr.append(similar)
        else:
            arr.append(0)
    H_matrix.append(arr)







print('time taken in generation {}'.format(time.time()-start))
start = time.time()
H_matrix = np.array(H_matrix)


print('time taken in saving H_matrix {}'.format(time.time() - start))

In [ ]:
type(w2v_model.vocab)

In [ ]:
print(H_matrix.dtype)
H_matrix = H_matrix.astype('float16')
np.save("H_matrix",H_matrix)
H_matrix = np.load('H_matrix.npy')

In [ ]:

print('data_matrix shape : {} and dtype of data : {}'.format(data_matrix.shape, data_matrix.dtype))
print('H_matrix shape : {} and dtype of H_matrix : {}'.format(H_matrix.shape, H_matrix.dtype))


In [ ]:
import time
start = time.time()
matrix_z = data_matrix.dot(H_matrix)
print('time taken in saving H_matrix {}'.format (time.time() - start))

In [ ]:
np.save('matrix_z_bbc_without_negSimilarity',matrix_z)

print('finally I get this matrix! huuu,,,')

In [ ]:
import numpy as np

In [ ]:
matrix_z = np.load('../input/mydata/matrix_z_bbc_without_negSimilarity.npy')
matrix_z = np.around(matrix_z,decimals=3)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error,confusion_matrix, precision_score, recall_score, auc,roc_curve, classification_report
from sklearn import ensemble, linear_model, neighbors, svm, tree, neural_network
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn import svm,model_selection, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data.replace(cleanup_nums,inplace = True)
cleanup_nums = {"category":     {"tech": 0, "business": 1,"sport":2, 'entertainment':3, 'politics':4}}
            

In [ ]:
# y = data['category']
X = matrix_z

In [ ]:
y = data['category']
y.unique()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X , y , test_size=0.2, random_state=38)

In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC
import time
start = time.time()
# defining parameter range 
param_grid = {'C': [0.001,0.01, 0.1],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['linear']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=4) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 

print('time taken in training SMV model is {}'.format(time.time()-start))
# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 


In [ ]:
grid_predictions = grid.predict(X_test)

print(classification_report(y_test, grid_predictions)) 
print(confusion_matrix(y_test,grid_predictions))

In [ ]:
tree_clf = tree.DecisionTreeClassifier()
start = time.time()
tree_clf.fit(X_train,y_train)
print('time taken in training tree_clf model is {}'.format(time.time()-start))
predictions_tree = tree_clf.predict(X_test)
print(confusion_matrix(y_test,predictions_tree))
print(classification_report(y_test,predictions_tree))
